# Network Creation

In [49]:
%load_ext autoreload
%autoreload

#%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

import networkx as nx
from networkTrips import organizeTrips
from timeUtils import clock, elapsed, getDateTime
from ioUtils import loadJoblib, saveFile, getFile
from fsUtils import mkDir, mkSubDir, setFile, setSubDir
from pandasUtils import getRowData, getColData, dropColumns
from geoUtils import convertMetersToLat, convertLatToMeters, convertMetersToLong, convertLongToMeters
from geocluster import geoClusters, geoCluster
from geoclusterUtils import genCenters, genCluster, genClusters, genTripsBetweenClusters
from networkClusterMaps import foliumMap
from networkOutput import printNetwork
from driverNetwork import driverNetwork
from networkFeatures import networkFeatures

import pandas as pd
pd.set_option("display.max_rows",1000)
pd.set_option('precision', 3)

import warnings
warnings.filterwarnings('ignore')

_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Current Time is Fri Dec 14, 2018 17:32:51 for Last Run


In [2]:
savedir = "/Users/tgadf/Downloads/network"
mkDir(savedir)

'/Users/tgadf/Downloads/network'

# Load/Generate Data

In [3]:
#######################################################################################
# Generate Clusted Data
#######################################################################################
genData = False
if genData:
    cls     = 20
    total   = 500
    genMax  = 75
    distMax = 500
    raw  = genClusters(cls, 250, latRange=[29.8, 30.2], lngRange=[49.8, 50.2], dist="gauss", maxrad=genMax)
    gc   = geoClusters(key="dummy", points=raw, distMax=distMax, debug=False)
    gc.findClusters(seedMin=2, debug=False)
    df   = genTripsBetweenClusters(n=total, gc=gc, returnDF=True)
    df["device"] = "dummy"    
    
    tmpdf = loadJoblib("/Users/tgadfort/Downloads/r4hIDs.p").sample(n=total, replace=True)
    tojoin = tmpdf.sample(cls)
    tojoin["cl"] = ["cl{0}".format(x) for x in range(cls)]

    df['cl'] = df['cl0']
    drops = [x for x in tojoin.columns if x.startswith("Geo1")]
    tojoinCL0 = dropColumns(tojoin, columns=drops, inplace=False)
    test = df.merge(tojoinCL0, on='cl')

    test['cl'] = test['cl1']
    drops = [x for x in tojoin.columns if x.startswith("Geo0")]
    tojoinCL1 = dropColumns(tojoin, columns=drops, inplace=False)
    test = test.merge(tojoinCL1, on='cl')

    gpsdata = test
    dropColumns(gpsdata, columns=["cl", "cl0", "cl1"])
    gpsdata.replace('nan', 0, inplace=True)
else:
    fname = "/Users/tgadf/Downloads/gpsTripsOakRidge (1).p"
    print("Loading {0}".format(fname))
    gpsdata = loadJoblib(fname)    

_, _ = clock("Last Run")

Loading /Users/tgadf/Downloads/gpsTripsOakRidge (1).p
Current Time is Fri Dec 14, 2018 13:00:04 for Last Run


## Show Data (if needed)

## Subselect (if needed)

In [4]:
device  = '352252060173789'
debug   = True
gpsdata = gpsdata[gpsdata['device'] == device]
print("Keeping {0} rows".format(gpsdata.shape[0]))
_, _ = clock("Last Run")

Keeping 3066 rows
Current Time is Fri Dec 14, 2018 13:00:04 for Last Run


# Cluster and Sort Trips

In [32]:
i  = 0
nd = gpsdata['device'].nunique() 
for device, df in gpsdata.groupby('device'):
    print('Key = {0}'.format(device),'\tRun = {0}/{1}'.format(i,nd),'\tTrips = {0}'.format(df.shape[0]))
    i += 1

    #######################################################################################
    # Cluster Geo Data (Lat, Long)
    #######################################################################################
    points         = df[["lat0", "long0"]]
    points.columns = ["lat", "long"]
    pnts           = df[["lat1", "long1"]]
    pnts.columns   = ["lat", "long"]    
    points         = points.append(pnts)



    #######################################################################################
    # Create Clusters
    #######################################################################################True
    gc   = geoClusters(key="dummy", points=points, distMax=200, mergeFactor=2.5, debug=False)
    gc.createCells(debug=False)
    gc.createProtoClusters(seedMin=4, debug=False)
    gc.createSeedlessClusters(seedMin=2, debug=False)
    gc.mergeClusters(debug=False)
    print("Found {0} geo clusters".format(gc.getNClusters()))



    #######################################################################################
    # Set Nearest Clusters
    #######################################################################################
    if debug:
        start, cmt = clock("Finding Nearest Clusters for Start of Trips")
    geoResults = df[['lat0', 'long0']].apply(gc.getNearestClusters, axis=1).values
    df["geo0"] = [x[0] for x in geoResults]
    if debug:
        elapsed(start, cmt)
        start, cmt = clock("Finding Nearest Clusters for End of Trips")
    geoResults = df[['lat1', 'long1']].apply(gc.getNearestClusters, axis=1).values
    df["geo1"] = [x[0] for x in geoResults]    
    if debug:
        elapsed(start, cmt)


    #######################################################################################
    # Organize Trips for Network
    #######################################################################################
    trips = organizeTrips(df=df, gc=gc, debug=True, requireGood=False)

Key = 352252060173789 	Run = 0/1 	Trips = 3066
Found 375 geo clusters
Current Time is Fri Dec 14, 2018 13:51:39 for Finding Nearest Clusters for Start of Trips
Current Time is Fri Dec 14, 2018 13:51:46 for Done with Finding Nearest Clusters for Start of Trips
Process [Done with Finding Nearest Clusters for Start of Trips] took 7 seconds.
Current Time is Fri Dec 14, 2018 13:51:46 for Finding Nearest Clusters for End of Trips
Current Time is Fri Dec 14, 2018 13:51:53 for Done with Finding Nearest Clusters for End of Trips
Process [Done with Finding Nearest Clusters for End of Trips] took 6 seconds.
All Trips:     3066
Deriving Home From Daily Visits, Overnight Stays, Dwell Times, and Common Location
There are 376 possible home clusters
There are 54 possible home clusters with at least two hours of dwell time
There are 38 possible home clusters with at least ten daily visits
There are 5 possible home clusters with overnight stays
Selecting cl0 as the home cluster with significance 109.8, 

## Investigate Individual Cluster

In [33]:
clname = 'cl9'
clusterData = df[(df['geo0'] == clname) | (df['geo1'] == clname)]

In [41]:
geocols = [x.replace("Geo0", "") for x in clusterData.columns if x.startswith("Geo0") and x.find("CENSUS") == -1 and x.find("ASDW") == -1]
from collections import Counter
cldata  = Counter()
clprefix = None
for ir,row in clusterData.iterrows():
    if row['geo0'] == clname:
        clprefix = "Geo0"
    elif row['geo1'] == clname:
        clprefix = "Geo1"    
    for g in range(2):
        for col in geocols:
            colname = "{0}{1}".format(clprefix, col)
            colval  = row[colname]
            try:
                if colval > 0:
                    cldata[col] += 1
            except:
                pass
            
tripcldata = {}
for k,v in trips['vertexMetrics'][clname].items():
    if not isinstance(v, dict):
        continue
    for k2,v2 in v.items():
        if isinstance(v2, Counter):
            if v2.get(1.0):
                tripcldata[k2] = v2
                        
### Show Results
print("Cluster {0} --> {1}".format(clname, cldata.most_common()))
print("Cluster {0} --> {1}".format(clname, tripcldata))

Cluster cl9 --> [('OSMParking', 200), ('OSMSchool', 8)]
Cluster cl9 --> {'OSMParking': Counter({1.0: 105, 0.0: 59}), 'OSMSchool': Counter({0.0: 160, 1.0: 4})}


# Saved Data

In [6]:
# Save trips/gc if needed
if False:
    deviceDir = mkSubDir(savedir, device)
    tripsfile = setFile(deviceDir, "trips.p")
    gcfile    = setFile(deviceDir, "gc.p")
    loadTrips=False
    if loadTrips:
        trips = getFile(tripsfile)
        gc    = getFile(gcfile)
    else:
        print("Saving to {0}".format(deviceDir))
        saveFile(ifile=gcfile, idata=gc)
        saveFile(ifile=tripsfile, idata=trips)
    
_, _ = clock("Last Run")

Current Time is Fri Dec 14, 2018 09:27:55 for Last Run


In [6]:
# Show data if needed
df.head().T

,950,951,4374,4375,4377
device,352252060173789,352252060173789,352252060173789,352252060173789,352252060173789
Start,2018-03-17 20:06:12,2018-02-01 15:56:11,2017-11-01 11:26:21,2017-09-08 15:12:41,2018-05-09 13:22:14
End,2018-03-17 20:08:00,2018-02-01 17:08:03,2017-11-01 11:32:45,2017-09-08 15:43:06,2018-05-09 13:46:36
total_miles,0.1,1.5,1.8,10.3,4.3
heading0,66,60,348,48,288
lat0,35.9,35.9,35.7,35.8,35.7
long0,-84.1,-84.2,-84.3,-84.3,-84.3
heading1,72,66,138,252,72
lat1,35.9,35.9,35.7,35.7,35.7
long1,-84.2,-84.2,-84.4,-84.4,-84.4


# Driver Network

In [56]:
%load_ext autoreload
%autoreload

from driverNetwork import driverNetwork
from edgeInfo import edgeInfo
from vertexInfo import vertexInfo
from networkCategories import categories
from networkAlgos import networkAlgos

start, cmt = clock("Running Driver Network")
dn = driverNetwork(trips)
dn.create(debug=True)
dn.computeNetworkAttrs(debug=True, level=1)
dn.fillVertexCensusData(debug=True)
dn.fillVertexGeospatialData(debug=True)
dn.fillVertexInternalData(debug=True)
dn.fillVertexNetworkData(debug=True)
dn.fillEdgeInternalData(debug=True)
dn.fillEdgeVertexData(debug=True)
dn.fillEdgeNetworkData(debug=True)
g = dn.getNetwork()
elapsed(start, cmt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Dec 14, 2018 17:53:53 for Running Driver Network
Creating a driver network with 376 vertices and 1027 edges.
Creating Network Attributes
Updating Vertex Attributes
Updating Vertices/Edges
Ordering Edges by Weight
Ordering Vertices by Centrality
Flattening Vertices/Edges
Flattening Vertex Attributes
Creating Vertex Attrs DataFrame
Cleaning Vertex Attribute Names
Creating Edge Attrs DataFrame
Cleaning Edge Attribute Names
Computing Network Attributes (simple)
Computing Network Algorithms
Running network algorithms
Creating Algorithm Results DataFrame for Vertices
Ranking Vertices
Creating Algorithm Results DataFrame for Edges
  Created 30 attributes for 376 vertices
  Created 2 attributes for 1027 edges
  Created 46 attributes for the network
Filling Vertex Census Data
Filling Vertex Geospatial Data
Filling Vertex Internal Data
Filling Vertex Network Data
Filling Edge Internal Data

In [57]:
if True:
    %load_ext autoreload
    %autoreload

    from networkOutput import printNetwork
    pn = printNetwork(dn)
    pn.printVertices(minN=40)
    pn.printEdges(minW=20)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


======================================== 376 Vertices (min 40) ========================================

#   Cl    N     Home Active    DayWeek   Dwell     Place                    Cliques Cluster Degree  Centrality  ShortPath   Spectral  PageRank  POIs                                              
--- ---   ---   ---  ---       ---       ---       ---                      ---     ---     ---     ---         ---         ---       ---       ---                                               
0   cl0   1721  ***  Daily     Week      VeryHigh  Loudon, Tennessee        53      73      266     188         210         306       118       Road                                              
1   cl1   587        Daily     Week      VeryLow   Loudon, Tennessee        181     73      57      10          369         373       301       Fastfood, Recreation, MajorRd, Road               
2   cl5   236        Week

In [48]:
dn.getVertexByName('cl9', 'attr')

DayOfWeek                                                               0
DrivingDistance                                                        25
GeoDistanceRatio                                                       25
N                                                                     164
First                                                          2017-07-19
Last                                                           2018-05-01
CoM                                                 (36.02193, -84.23355)
Radius                                                                 98
Cells                                                                  25
Quantiles                                            [19, 47, 67, 81, 98]
Geohashs                [dnkhgjwc, dnkhgjwd, dnkhgjwb, dnkhgjwe, dnkhg...
DwellTime                                                            3.55
DailyVisits                                                            61
OvernightStays                        

# Network Features

In [51]:
%load_ext autoreload
%autoreload

from networkFeatures import networkFeatures
from networkAlgos import algos

nf = networkFeatures(dn)

## Vertex Counts
nf.fillVertexCensusCounts(debug=True)
nf.fillVertexInternalCounts(debug=True)
nf.fillVertexGeoSpatialCounts(debug=True)
nf.fillVertexProperties(debug=True)

## Edge Counts
nf.fillEdgeInternalCounts(debug=True)
nf.fillEdgeCensusCounts(debug=True)
nf.fillEdgeGeoSpatialCounts(debug=True)
nf.fillEdgeProperties(debug=True)

## Network Counts
nf.fillNetworkFeatures(debug=True)

## Home Counts
nf.fillHomeFeatures(debug=True)

## Indiv Vertex/Edge Values
#nf.fillIndividualVertexFeatures(debug=True)
#nf.fillIndividualEdgeFeatures(debug=True)

## Vertex/Edge Correlations
##nf.fillVertexFeatureCorrelations(debug=True)
#nf.fillEdgeFeatureCorrelations(debug=True)
_,_ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Filling Vertex Census Counts
  Filling Vertex Census Counts
Filling Vertex Internal Counts
  Filling Vertex Internal Counts for 34 Cutoff Values
Filling Vertex GeoSpatial Counts
  Filling Vertex GeoSpatial Counts for 0 Cutoff Values
Filling Vertex Properties
  Filled Vertex Properties for 43 Attributes
Filling Edge Internal Counts
  Filling edge Internal Counts for 29 Cutoff Values
Filling Edge Census Counts
  Filling Edge Census Counts
Filling edge GeoSpatial Counts
  Filling edge GeoSpatial Counts for 106 Cutoff Values
Filling Edge Properties
  Filled Edge Properties for 11 Attributes
Filling Network Features
  Filled 46 Network Features
Filling Home Vertex Features
  Filled 178 Home Vertex Features
Current Time is Wed Dec 12, 2018 13:04:50 for Last Run


In [ ]:
nf.getFeatures()

In [42]:
nf.getHomeFeatureDataFrame().T

,0
HomeRatio,High
HomeDays,Long
HomeRank,0
HomeCensusCbsa,"Knoxville, TN"
HomeCensusCbsaType,Metro
HomeCensusCbsaPop,Big
HomeCensusCbsaHousing,Big
HomeCensusCbsaArea,Mid
HomeCensusCsa,"Knoxville-Sevierville-La Follette, TN"
HomeCensusCsaType,CSA


# GeoSpatial Maps

In [59]:
from folium import PolyLine, CircleMarker, Circle, Marker, Icon, FeatureGroup, Map, LayerControl
import geohash


class foliumMap():
    def __init__(self, df=None, pc=None, gc=None, dn=None, nf=None):
        self.pc = None
        if pc is not None:
            self.setProtoClusters(pc)
        self.gc = None
        if gc is not None:
            self.setGeoClusters(gc)
        self.dn = None
        if dn is not None:
            self.setDriverNetwork(dn)
        self.nf = None
        if nf is not None:
            self.setNetworkFeatures(nf)
        self.df = None
        if df is not None:
            self.setTripsDataFrame(df)
        
        self.m  = None
        
        self.init_zoom = 10
        
        self.colors = ['red', 'blue', 'gray', 'darkred', 'lightred', 'orange', 'beige', 'green', 'darkgreen', 'lightgreen',
                       'darkblue', 'lightblue', 'purple', 'darkpurple', 'pink', 'cadetblue', 'lightgray']
        
        
    ########################################################################################
    # Setters
    ########################################################################################
    def setTripsDataFrame(self, df):
        self.df         = df
        self.df         = df[["lat0", "long0"]]
        self.df.columns = ["lat", "long"]
        pnts            = df[["lat1", "long1"]]
        pnts.columns    = ["lat", "long"]    
        self.df         = self.df.append(pnts)
        
    def setProtoClusters(self, pc):
        self.pc = pc
        
    def setGeoClusters(self, gc):
        self.gc = gc
        
    def setDriverNetwork(self, dn):
        self.dn = dn
        
    def setNetworkFeatures(self, nf):
        self.nf = nf
        
        
        
    ########################################################################################
    # Getters
    ########################################################################################        
    def getMap(self):
        return self.m
        
        
        
    ########################################################################################
    ########################################################################################
    # Create Map
    ########################################################################################
    ########################################################################################
    def createMapFromTripsDataFrame(self, zoom=None, debug=False):
        if self.df is None:
            print("There is no trips DataFrame object!")
            return
        
        try:
            lat0           = self.df['lat'].mean()
            long0          = self.df['long'].mean()
        except:
            raise ValueError("Could not get center of geo clusters and create map!")
            
        if zoom is None:
            zoom = self.init_zoom
        self.m = Map(location=[lat0, long0], zoom_start=zoom)
        
        
    def createMapFromProtoClusters(self, zoom=None, debug=False):
        if self.pc is None:
            print("There is no ProtoClusters object!")
            return
        
        try:
            lats  = Series(x["CoM"][0] for pcl,x in self.pc.items())
            lngs  = Series(x["CoM"][1] for pcl,x in self.pc.items())
            lat0  = lats.mean()
            long0 = lngs.mean()
        except:
            raise ValueError("Could not get center of geo clusters and create map!")
            
        if zoom is None:
            zoom = self.init_zoom
        self.m = Map(location=[lat0, long0], zoom_start=zoom)
        
        
    def createMapFromGeoClusters(self, zoom=None, debug=False):
        if self.gc is None:
            print("There is no GeoClusters object!")
            return
        
        try:
            coms  = self.gc.getClusterCoMs()
            lat0  = coms[0].mean()
            long0 = coms[1].mean()
        except:
            raise ValueError("Could not get center of geo clusters and create map!")
            
        if zoom is None:
            zoom = self.init_zoom
        self.m = Map(location=[lat0, long0], zoom_start=zoom)
        
        
    def createMapFromDriverNetwork(self, zoom=None, debug=False):
        if self.dn is None:
            print("There is no DriverNetwork object!")
            return

        if self.gc is None:
            try:
                self.dn.createGC()            
                self.gc = self.dn.getGC()
            except:
                raise ValueError("Could not get create geo clusters from driver network")

        try:
            coms  = self.gc.getClusterCoMs()
            lat0  = coms[0].mean()
            long0 = coms[1].mean()
        except:
            raise ValueError("Could not get center of geo clusters and create map!")
        
        
        
        if zoom is None:
            zoom = self.init_zoom
        self.m = Map(location=[lat0, long0], zoom_start=zoom)
        
        
    def createMapFromNetworkFeatures(self, zoom=None, debug=False):
        if self.nf is None:
            print("There is no NetworkFeatures object!")
            return
        
        if zoom is None:
            zoom = self.init_zoom
        self.m = Map(location=[lat0, long0], zoom_start=zoom)
        
        
    def createMap(self, zoom=None, debug=False):
        if self.nf is not None:
            self.createMapFromNetworkFeatures(zoom=zoom, debug=debug)
        elif self.dn is not None:
            self.createMapFromDriverNetwork(zoom=zoom, debug=debug)
        elif self.gc is not None:
            self.createMapFromGeoClusters(zoom=zoom, debug=debug)
        elif self.pc is not None:
            self.createMapFromProtoClusters(zoom=zoom, debug=debug)
        elif self.df is not None:
            self.createMapFromTripsDataFrame(zoom=zoom, debug=debug)
        else:
            raise ValueError("Cannot create map because there is no data object!")
            
        
        
    ########################################################################################
    ########################################################################################
    # Points/Clusters
    ########################################################################################
    ########################################################################################      
    def addPointsFromTripsDataFrame(self, debug=False):
        if self.m is None:
            print("Folium Map is None!")
            return
        
        if self.df is None:
            print("DataFrame is None!")
            return
        
        cols = ['darkblue', 'lightblue', 'pink', 'lightgray']
        
        rad = 5
        weight = 1
        for row in self.df.iterrows():
            com = row[1].values
            Circle(com, color=cols[1], radius=rad, fill=True, fill_color=cols[0], weight=weight, opacity=0).add_to(self.m)
    
    def addPointsFromGeoClusterCells(self, debug=False):
        if self.m is None:
            print("Folium Map is None!")
            return
        
        if self.gc is None:
            print("GeoClusters is None!")
            return

        cells = gc.getCells()
        for geo,cnt in cells.iteritems():
            com    = geohash.decode_exactly(geo)[:2]
            wgt    = int(cnt)
            popup  = str(wgt)
            Circle(com, color='black', radius=5, fill=True, fill_color='black', weight=wgt, opacity=0, popup=popup).add_to(self.m)
    
    
    def addPointsFromProtoClusters(self, debug=False):
        if self.m is None:
            print("Folium Map is None!")
            return
        
        if self.gc is None:
            print("GeoClusters is None!")
            return
        
        cols = ['darkblue', 'lightblue', 'pink', 'lightgray']
        
        from pandas import Series

        for pcl,protoCluster in gc.getProtoClusters().items():
            com    = protoCluster["CoM"]
            rad    = max([protoCluster["Radius"], 5])
            counts = protoCluster["Counts"]
            weight = int(counts)
            name   = pcl
            popup  = "{0} : N = {1}".format(name, counts)

            Circle(com, color=cols[0], radius=rad, fill=True, fill_color=cols[0], weight=weight, opacity=0).add_to(self.m)
    
    
    def addPointsFromSeedlessClusters(self, debug=False):
        if self.m is None:
            print("Folium Map is None!")
            return
        
        if self.gc is None:
            print("GeoClusters is None!")
            return
        
        cols = ['darkgreen']
        
        from pandas import Series

        for scl,seedlessCluster in gc.getSeedlessClusters().items():
            com    = seedlessCluster["CoM"]
            rad    = max([seedlessCluster["Radius"], 5])
            counts = seedlessCluster["Counts"]
            weight = int(counts)
            name   = scl
            popup  = "{0} : N = {1}".format(name, counts)

            Circle(com, color=cols[0], radius=rad, fill=True, fill_color=cols[0], weight=weight, opacity=0).add_to(self.m)
    
    
    def addPointsFromMergedClusters(self, debug=False):
        if self.m is None:
            print("Folium Map is None!")
            return
        
        if self.gc is None:
            print("GeoClusters is None!")
            return
        
        cols = ['darkred']
        
        from pandas import Series

        for cl,cluster in gc.getMergedClusters().items():
            com    = cluster["CoM"]
            rad    = max([cluster["Radius"], 5])
            counts = cluster["Counts"]
            weight = int(counts)
            name   = cl
            popup  = "{0} : N = {1}".format(name, counts)

            Circle(com, color=cols[0], radius=rad, fill=True, fill_color=cols[0], weight=weight, opacity=0).add_to(self.m)
    
    
    def addPointsFromGeoClusters(self, debug=False):
        if self.m is None:
            print("Folium Map is None!")
            return
        
        if self.gc is None:
            print("GeoClusters is None!")
            return
        
        cols = ['darkblue', 'lightblue', 'pink', 'lightgray']
        
        from pandas import Series
        feature_group_1 = FeatureGroup(name="Driver Top 90%")
        feature_group_2 = FeatureGroup(name="Driver Top 75%")
        feature_group_3 = FeatureGroup(name="Driver Top 50%")
        feature_group_4 = FeatureGroup(name="Driver Low 50%")

        weights = Series([cluster.getCounts() for cl,cluster in gc.getClusters().items()])
        alpha   = weights.quantile(0.9)
        beta    = weights.quantile(0.75)
        gamma   = weights.quantile(0.5)

        for cl,cluster in gc.getClusters().items():
            com    = cluster.getCoM()
            rad    = max([cluster.getRadius(), 10])
            counts = cluster.getCounts()
            weight = float(counts)
            quant  = cluster.getQuantiles()
            name   = cl
            cells  = ",".join(cluster.getCells())
            popup  = "{0} : N = {1} : {2}".format(name, weight, com)
            #popup  = ""

            if counts > alpha:
                Marker(com, icon=Icon(color=cols[0], icon_color='white', icon="car", angle=0, prefix='fa'), popup=popup).add_to(feature_group_1)
                Circle(com, color=cols[0], radius=rad, fill=True, fill_color=cols[0], weight=weight, opacity=0).add_to(feature_group_1)
            elif counts > beta:
                Marker(com, icon=Icon(color=cols[1], icon_color='white', icon="car", angle=0, prefix='fa'), popup=popup).add_to(feature_group_2)
                Circle(com, color=cols[1], radius=rad, fill=True, fill_color=cols[1], weight=weight, opacity=0).add_to(feature_group_2)
            elif counts > gamma:
                Marker(com, icon=Icon(color=cols[2], icon_color='white', icon="car", angle=0, prefix='fa'), popup=popup).add_to(feature_group_3)
                Circle(com, color=cols[2], radius=rad, fill=True, fill_color=cols[2], weight=weight, opacity=0).add_to(feature_group_3)
            else:
                Marker(com, icon=Icon(color=cols[3], icon_color='white', icon="car", angle=0, prefix='fa'), popup=popup).add_to(feature_group_4)
                Circle(com, color=cols[3], radius=rad, fill=True, fill_color=cols[3], weight=weight, opacity=0).add_to(feature_group_4)

        feature_group_1.add_to(self.m)
        feature_group_2.add_to(self.m)
        feature_group_3.add_to(self.m)
        feature_group_4.add_to(self.m)
        LayerControl().add_to(self.m)
        
        
     
    def addPointsFromDriverNetwork(self, debug=False):
        if self.m is None:
            print("Folium Map is None!")
            return
        
        if self.dn is None:
            print("DrivingNetwork is None!")
            return
        
        if self.gc is None:
            print("GeoClusters is None!")
            return
        
        cols = ['darkblue', 'lightblue', 'pink', 'lightgray']
        
        from pandas import Series
        feature_group_1 = FeatureGroup(name="Driver Home")
        feature_group_2 = FeatureGroup(name="Daily Visits")
        feature_group_3 = FeatureGroup(name="Weekly Visits")
        feature_group_4 = FeatureGroup(name="Monthly Visits")
        feature_group_5 = FeatureGroup(name="Infrequent Visits")
        feature_group_5 = None

        clusters = self.gc.getClusters()

        for vertexName in self.dn.getVertices():
            if vertexName == 'None':
                continue
            cluster = clusters[vertexName]
            clname  = cluster.getName()
            com     = cluster.getCoM()
            rad     = max([int(cluster.getRadius()), 10])
            
            weight = 10
            clusterFeatures = self.dn.getVertexByName(vertexName, "feat")
            home   = clusterFeatures["Internal"]["IsHome"]
            place  = clusterFeatures["Census"]["Place"]
            active = clusterFeatures["Internal"]["FractionalVisits"]
            visits = clusterFeatures["Internal"]["DailyVisits"]
            n      = int(clusterFeatures["Internal"]["N"])
            pois   = ", ".join([k for k,v in clusterFeatures["GeoSpatial"].items() if v > 0])
            popup = "{0} ({1}) : N = {2} : fActive = {3} : POIs: {4} : {5}".format(vertexName, place, n, active, pois, com)
            #print(vertexName,popup)

            if home == 1:
                Marker(com, icon=Icon(color='darkred', icon_color='white', icon="home", angle=0, prefix='fa'), popup=popup).add_to(feature_group_1)
                Circle(com, color='darkred', radius=rad, fill=True, fill_color='darkred', weight=weight, opacity=0).add_to(feature_group_1)
            elif active == "Daily":
                Marker(com, icon=Icon(color=cols[0], icon_color='white', icon="car", angle=0, prefix='fa'), popup=popup).add_to(feature_group_2)
                Circle(com, color=cols[0], radius=rad, fill=True, fill_color=cols[0], weight=weight, opacity=0).add_to(feature_group_2)
            elif active == "Weekly":
                Marker(com, icon=Icon(color=cols[1], icon_color='white', icon="car", angle=0, prefix='fa'), popup=popup).add_to(feature_group_3)
                Circle(com, color=cols[1], radius=rad, fill=True, fill_color=cols[1], weight=weight, opacity=0).add_to(feature_group_3)
            elif active == "Monthly":
                Marker(com, icon=Icon(color=cols[2], icon_color='white', icon="car", angle=0, prefix='fa'), popup=popup).add_to(feature_group_4)
                Circle(com, color=cols[2], radius=rad, fill=True, fill_color=cols[2], weight=weight, opacity=0).add_to(feature_group_4)
            else:
                if feature_group_5 is not None:
                    Marker(com, icon=Icon(color=cols[3], icon_color='white', icon="car", angle=0, prefix='fa'), popup=popup).add_to(feature_group_5)
                    Circle(com, color=cols[3], radius=rad, fill=True, fill_color=cols[3], weight=weight, opacity=0).add_to(feature_group_5)

        feature_group_1.add_to(self.m)
        feature_group_2.add_to(self.m)
        feature_group_3.add_to(self.m)
        feature_group_4.add_to(self.m)
        if feature_group_5 is not None:
            feature_group_5.add_to(self.m)
        LayerControl().add_to(self.m)        
        
           
    def addPoints(self, debug=False):
        if self.m is None:
            raise ValueError("Cannot add points to an empty map")
        if self.nf is not None:
            self.addPointsFromNetworkFeatures(debug=debug)
        elif self.dn is not None:
            self.addPointsFromDriverNetwork(debug=debug)
        elif self.pc is not None:
            self.addPointsFromProtoClusters(debug=debug)
        elif self.gc is not None:
            self.addPointsFromGeoClusters(debug=debug)
        elif self.df is not None:
            self.addPointsFromTripsDataFrame(debug=debug)
        else:
            print("Cannot add points because there is map and object!")

## Show Clustering GeoSpatial Map

In [12]:
fm = foliumMap(pc=gc.getProtoClusters(), gc=gc)
fm.createMap(zoom=11)
fm.addPointsFromGeoClusterCells()
fm.addPointsFromGeoClusters()
m = fm.getMap()
m.save("/Users/tgadf/Downloads/oak-ridge-map.html")
#m

## Show Driver Network GeoSpatial Map

In [60]:
fm = foliumMap(dn=dn, gc=gc)
fm.createMap(zoom=11)
fm.addPointsFromDriverNetwork()
m = fm.getMap()
#m.save("oak-ridge-map-dn2.html")
m

In [ ]:


fm = foliumMap(pc=gc.getProtoClusters(), gc=gc)
fm.createMap()
fm.addPointsFromGeoClusterCells()
fm.addPointsFromProtoClusters()
m = fm.getMap()
m

fm = foliumMap(df=df)
fm.createMap()
fm.addPoints()
m = fm.getMap()
m.save("oak-ridge-map-df2.html")

fm = foliumMap(gc=gc, df=df)
fm.createMap()
fm.addPoints()
fm.addPointsFromTripsDataFrame()
m = fm.getMap()
m.save("oak-ridge-map-gc2.html")

In [79]:
%load_ext autoreload
%autoreload

from networkOutput import printNetwork
pn = printNetwork(dn)
pn.printVertices(minN=40)
pn.printEdges(minW=20)
pn.printFrequencies()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


======================================== 377 Vertices (min 40) ========================================

#   Cl    N     Home Active    DayWeek   Dwell     Place                    Cliques Cluster Degree  Centrality  ShortPath   Spectral  PageRank  POIs                                              
--- ---   ---   ---  ---       ---       ---       ---                      ---     ---     ---     ---         ---         ---       ---       ---                                               
0   cl0   1721  ***  Daily     Week      VeryHigh  Loudon, Tennessee        182     73      57      10          370         373       302       Road                                              
1   cl1   587        Daily     Week      VeryLow   Loudon, Tennessee        274     243     268     293         72          231       219       MajorRd, Road                                     
2   cl5   236        Week

In [80]:
dn.getVertexByName('cl9')

{'DayOfWeek': {'Avg': 0.0, 'Std': 0.0},
 'DrivingDistance': {'Avg': 25.01993706722832, 'Std': 35.02669631707319},
 'GeoDistanceRatio': {'Avg': 25.01993706722832, 'Std': 35.02669631707319},
 'N': 164,
 'First': datetime.date(2017, 7, 19),
 'Last': datetime.date(2018, 5, 1),
 'CoM': (36.02193, -84.23355),
 'Radius': 98.0,
 'Cells': 25,
 'Quantiles': [19, 47, 67, 81, 98],
 'Geohashs': ['dnkhgjwc',
  'dnkhgjwd',
  'dnkhgjwb',
  'dnkhgjwe',
  'dnkhgjwg',
  'dnkhgjx7',
  'dnkhgjqx',
  'dnkhgjqz',
  'dnkhgjrp',
  'dnkhgjx1',
  'dnkhgjxj',
  'dnkhgjwv',
  'dnkhgjwy',
  'dnkhgjxn',
  'dnkhgjxq',
  'dnkhgjwu',
  'dnkhgjxh',
  'dnkhgjxk',
  'dnkhgjxs',
  'dnkhgjxp',
  'dnkhgjxr',
  'dnkhgjxm',
  'dnkhgjw3',
  'dnkhgjw7',
  'dnkhgjqy'],
 'DwellTime': 3.548336720867208,
 'DailyVisits': 61,
 'OvernightStays': 0,
 'IsHome': 0,
 'Interval': 287,
 'FractionalActive': 0.46818923327895595,
 'FractionalVisits': 0.09951060358890701,
 'CENSUSCbsa': Counter({28940: 1}),
 'CENSUSCsa': Counter({314: 1}),
 'CEN

In [55]:
vertexData = dn.getVertexByName('cl0', 'feat')

In [56]:
vertexData

{'Census': {'Cbsa': 'Knoxville, TN',
  'CbsaType': 'Metro',
  'CbsaPop': 'Big',
  'CbsaHousing': 'Big',
  'CbsaArea': 'Mid',
  'Csa': 'Knoxville-Sevierville-La Follette, TN',
  'CsaType': 'CSA',
  'CsaPop': 'Mid',
  'CsaHousing': 'Mid',
  'CsaArea': 'Mid',
  'Metdiv': None,
  'MetdivType': 'Other',
  'MetdivPop': 'RealSmall',
  'MetdivHousing': 'RealSmall',
  'MetdivArea': 'RealSmall',
  'Place': 'Loudon',
  'PlaceType': 'Town',
  'PlacePop': 'Mid',
  'PlaceHousing': 'Mid',
  'PlaceArea': 'Small',
  'County': 'Loudon County',
  'CountyType': 'A',
  'CountyPop': 'Mid',
  'CountyHousing': 'Mid',
  'CountyArea': 'Small',
  'State': 'Tennessee',
  'StateRegion': 'SOUTHEastSouthCentral',
  'Region': 'SOUTHEastSouthCentral'},
 'GeoSpatial': {'Attraction': 0,
  'Auto': 0,
  'Building': 0,
  'College': 0,
  'Commercial': 0,
  'Cycling': 0,
  'Entertainment': 0,
  'Fastfood': 0,
  'Fuel': 0,
  'Grocery': 0,
  'Industrial': 0,
  'Lodging': 0,
  'Medical': 0,
  'Municipal': 0,
  'Parking': 0,
  '